# Chapter 4. 텍스트와 바이트

## 4.1 문자 문제

- '문자열'의 개념은 말 그대로 '문자의 열'이지만 '문자'의 정의가 모호
- 현재 '문자'를 가장 잘 정의한 것은 **유니코드**
- 파이썬 3의 `str`에서도 **유니코드** 문자를 사용

- 문자의 **코드 포인트**는 10진수 0에서 1,114,111까지의 숫자이며, 표준에 따라 **U+** 접두사를 붙여 4자리~6자리 사이의 16진수로 표현
- e.g. A의 코드 포인트는 U+0041, 유로 기호의 코드 포인트는 U+20AC
- 이러한 문자를 표현하는 실제 바이트는 **인코딩** 기법에 따라 달라짐

- 여기서 코드 포인트를 바이트로 변환하는 것을 **인코딩**, 바이트를 코드 포인트로 변환하는 것을 **디코딩**이라 함

In [7]:
s = 'café'
len(s)

4

In [8]:
b = s.encode('utf8')
b

b'caf\xc3\xa9'

In [9]:
len(b)

5

In [10]:
b.decode('utf8')

'café'

## 4.2 바이트에 대한 기본 지식

- 새로 도입된 이진 시퀀스형은 파이썬 2의 `str`과 다른 자료형
- 이진 시퀀스를 위해 사용되는 내장 자료형은 `bytes`와 `bytearray`
    - `bytes` 형은 파이썬 3에서 소개된 불변형
    - `bytearray` 형은 파이썬 2.6에서 추가된 가변형
- `bytes`와 `bytearray`에 들어 있는 각 항목은 0에서 255 사이의 정수
- 이진 시퀀스를 슬라이싱하면 언제나 동일한 자료형의 이진 시퀀스가 만들어짐

In [11]:
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

In [12]:
cafe[0]

99

In [16]:
cafe[1]

97

In [13]:
cafe[:1]  # bytes는 슬라이싱 해도 bytes

b'c'

In [14]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [15]:
cafe_arr[-1:]  # bytearray는 슬라이싱해도 bytearray

bytearray(b'\xa9')

이진 시퀀스는 각 바이트 값에 따라 다음과 같이 출력
- 화면에 출력 가능한 아스키 문자는 아스키 문자 그대로 출력
- 탭, 개행 문자, 캐리지 리턴, 백슬래시는 이스케이프 시퀀스로 출력
- 그 외 값은 널 바이트를 나타내는 16진수 이스케이스 시퀀스로 출력

아래 예제에서 처음 3바이트 b'caf'는 출력 가능한 범위이며, 나머지 2바위트는 그렇지 않기 때문에 이스케이프 시퀀스로 출력

In [17]:
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

- `bytes`와 `bytearray`는 포매팅하는 `format()`과 `format_map()` 메서드를 제외하고는 `str`이 제공하는 메서드를 모두 지원하며, `casefold()`, `isdecimal()`, `isidentifier()` 등 유니코드 데이터에 관련된 메서드를 지원
- 따라서 `endswith()`, `replace()`, `strip()` 등의 메서드를 `bytes` 인수에도 적용 가능
- 또한 `re` 모듈에서 제공하는 정규 표현식을 이진 시퀀스에도 적용 가능
- `fromhex()` 메서드는 공백으로 구분된 16진수 쌍을 파싱해 이진 시퀀스 생성 가능

In [19]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

버퍼 등의 객체로부터 이진 시퀀스를 생성하는 방법은 **저수준 연산**으로, 형변환이 필요할 수 있음

In [21]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])  # short int (16 bits)
octets = bytes(numbers)
octets  # 다섯개의 short int 형을 나타내는 10 bytes

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

- 버퍼와 같은 객체로부터 `bytes`나 `bytearray` 객체를 생성하면 언제나 바이트를 복사
- 이와 반대로 `memoryview`는 이진 데이터 구조체 간 메모리를 공유
- 이진 시퀀스에서 구조화된 정보를 추출하기 위해서는 `struct` 모듈 사용

### 4.2.1 구조체와 메모리 뷰
- `struct` 모듈은 패킹된 바이트를 다양한 형의 필드로 구성된 튜플로 분석
- 반대로 튜플은 패킹된 바이트로 변환하는 함수를 제공
- `memoryview` 클래스로 바이트 시퀀스를 생성하거나 저장할 수는 없지만, 바이트를 복사하지 않고 다른 이진 시퀀스, 패킹 배열, 혹은 PIL 이미지 등 버퍼 데이터의 슬라이스에 **공유 메모리 방식**으로 접근할 수 있게 해줌

In [22]:
import struct
fmt = '<3s3sHH'
with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read())

FileNotFoundError: [Errno 2] No such file or directory: 'filter.gif'

In [ ]:
header = img[:10]
bytes(header)

struct.unpack(fmt, header)
del header
del img

## 4.3 기본 인코더/디코더

- 파이썬에는 100여 개의 **코덱 (인코더/디코더)**이 존재
- 코덱은 `open()`, `str.encode()`, `bytes.decode()` 등의 함수를 호출할 때 `encoding` 인수에 전달해서 사용 가능

In [23]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## 4.4 인코딩/디코딩 문제 이해하기

### 4.4.1 UnicodeEncodeError 처리하기

